# QR Algorithm for Eigenvalue Computation

## Introduction

The **QR algorithm** is one of the most important numerical methods for computing the eigenvalues of a matrix. Developed independently by John G.F. Francis and Vera Kublanovskaya in 1961, it remains the foundation of modern eigenvalue solvers.

## Mathematical Foundation

### Eigenvalue Problem

For a square matrix $\mathbf{A} \in \mathbb{R}^{n \times n}$, the eigenvalue problem seeks to find scalar $\lambda$ and non-zero vector $\mathbf{v}$ such that:

$$\mathbf{A}\mathbf{v} = \lambda\mathbf{v}$$

The eigenvalues $\lambda_1, \lambda_2, \ldots, \lambda_n$ are the roots of the characteristic polynomial:

$$\det(\mathbf{A} - \lambda\mathbf{I}) = 0$$

### QR Decomposition

The QR decomposition factorizes a matrix $\mathbf{A}$ into:

$$\mathbf{A} = \mathbf{Q}\mathbf{R}$$

where:
- $\mathbf{Q}$ is an orthogonal matrix ($\mathbf{Q}^T\mathbf{Q} = \mathbf{I}$)
- $\mathbf{R}$ is an upper triangular matrix

### The Basic QR Algorithm

The QR algorithm iteratively transforms the matrix toward a form where eigenvalues are easily extracted:

1. Set $\mathbf{A}_0 = \mathbf{A}$
2. For $k = 0, 1, 2, \ldots$:
   - Compute QR decomposition: $\mathbf{A}_k = \mathbf{Q}_k\mathbf{R}_k$
   - Form the next iterate: $\mathbf{A}_{k+1} = \mathbf{R}_k\mathbf{Q}_k$

### Key Theorem

Under suitable conditions, the sequence $\{\mathbf{A}_k\}$ converges to a quasi-triangular matrix (Schur form). For a real matrix with distinct real eigenvalues, this is an upper triangular matrix with eigenvalues on the diagonal.

### Similarity Transformation

Each iteration preserves eigenvalues because $\mathbf{A}_{k+1}$ is similar to $\mathbf{A}_k$:

$$\mathbf{A}_{k+1} = \mathbf{R}_k\mathbf{Q}_k = \mathbf{Q}_k^T\mathbf{Q}_k\mathbf{R}_k\mathbf{Q}_k = \mathbf{Q}_k^T\mathbf{A}_k\mathbf{Q}_k$$

### Convergence Rate

For distinct eigenvalues $|\lambda_1| > |\lambda_2| > \cdots > |\lambda_n|$, the subdiagonal elements converge to zero at rate:

$$|a_{i+1,i}^{(k)}| = O\left(\left|\frac{\lambda_{i+1}}{\lambda_i}\right|^k\right)$$

### QR Algorithm with Shifts

To accelerate convergence, we introduce a **shift** $\mu_k$:

1. $\mathbf{A}_k - \mu_k\mathbf{I} = \mathbf{Q}_k\mathbf{R}_k$
2. $\mathbf{A}_{k+1} = \mathbf{R}_k\mathbf{Q}_k + \mu_k\mathbf{I}$

The **Rayleigh quotient shift** uses $\mu_k = a_{nn}^{(k)}$ (bottom-right element), achieving cubic convergence for symmetric matrices.

## Implementation

We will implement:
1. Basic QR algorithm without shifts
2. QR algorithm with Rayleigh quotient shift
3. Visualization of convergence behavior

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import qr, norm

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully")

In [ ]:
def qr_algorithm_basic(A, max_iterations=100, tol=1e-10):
    """
    Basic QR algorithm for eigenvalue computation.
    
    Parameters:
    -----------
    A : ndarray
        Square matrix
    max_iterations : int
        Maximum number of iterations
    tol : float
        Convergence tolerance for off-diagonal elements
    
    Returns:
    --------
    eigenvalues : ndarray
        Computed eigenvalues (diagonal of final matrix)
    history : list
        History of diagonal elements at each iteration
    off_diagonal_norm : list
        History of off-diagonal norm for convergence tracking
    """
    n = A.shape[0]
    A_k = A.copy().astype(float)
    
    history = [np.diag(A_k).copy()]
    off_diagonal_norm = []
    
    for k in range(max_iterations):
        # QR decomposition
        Q, R = qr(A_k)
        
        # Form next iterate
        A_k = R @ Q
        
        # Track convergence
        history.append(np.diag(A_k).copy())
        
        # Compute off-diagonal norm (lower triangular part)
        off_diag = np.tril(A_k, -1)
        off_norm = norm(off_diag, 'fro')
        off_diagonal_norm.append(off_norm)
        
        # Check convergence
        if off_norm < tol:
            print(f"Basic QR converged in {k+1} iterations")
            break
    else:
        print(f"Basic QR did not converge in {max_iterations} iterations")
    
    eigenvalues = np.diag(A_k)
    return eigenvalues, history, off_diagonal_norm

In [ ]:
def qr_algorithm_shifted(A, max_iterations=100, tol=1e-10):
    """
    QR algorithm with Rayleigh quotient shift for accelerated convergence.
    
    Parameters:
    -----------
    A : ndarray
        Square matrix (should be symmetric for guaranteed convergence)
    max_iterations : int
        Maximum number of iterations
    tol : float
        Convergence tolerance
    
    Returns:
    --------
    eigenvalues : ndarray
        Computed eigenvalues
    history : list
        History of diagonal elements
    off_diagonal_norm : list
        History of off-diagonal norm
    """
    n = A.shape[0]
    A_k = A.copy().astype(float)
    
    history = [np.diag(A_k).copy()]
    off_diagonal_norm = []
    
    for k in range(max_iterations):
        # Rayleigh quotient shift (bottom-right element)
        mu = A_k[-1, -1]
        
        # Shifted QR decomposition
        Q, R = qr(A_k - mu * np.eye(n))
        
        # Form next iterate with shift added back
        A_k = R @ Q + mu * np.eye(n)
        
        # Track convergence
        history.append(np.diag(A_k).copy())
        
        # Compute off-diagonal norm
        off_diag = np.tril(A_k, -1)
        off_norm = norm(off_diag, 'fro')
        off_diagonal_norm.append(off_norm)
        
        # Check convergence
        if off_norm < tol:
            print(f"Shifted QR converged in {k+1} iterations")
            break
    else:
        print(f"Shifted QR did not converge in {max_iterations} iterations")
    
    eigenvalues = np.diag(A_k)
    return eigenvalues, history, off_diagonal_norm

## Test Case: Symmetric Matrix

We create a symmetric matrix to demonstrate the algorithm. Symmetric matrices are guaranteed to have real eigenvalues and provide well-behaved convergence.

In [ ]:
# Create a symmetric test matrix
n = 5
B = np.random.randn(n, n)
A = (B + B.T) / 2  # Symmetrize

print("Test Matrix A:")
print(A)
print(f"\nMatrix size: {n}x{n}")
print(f"Is symmetric: {np.allclose(A, A.T)}")

In [ ]:
# Compute eigenvalues using both methods
print("="*50)
print("Basic QR Algorithm")
print("="*50)
eig_basic, hist_basic, off_basic = qr_algorithm_basic(A, max_iterations=200)

print("\n" + "="*50)
print("Shifted QR Algorithm (Rayleigh Quotient)")
print("="*50)
eig_shifted, hist_shifted, off_shifted = qr_algorithm_shifted(A, max_iterations=200)

# Reference eigenvalues from NumPy
eig_numpy = np.linalg.eigvalsh(A)

print("\n" + "="*50)
print("Eigenvalue Comparison")
print("="*50)
print(f"\nNumPy (reference):     {np.sort(eig_numpy)}")
print(f"Basic QR (sorted):     {np.sort(eig_basic)}")
print(f"Shifted QR (sorted):   {np.sort(eig_shifted)}")

In [ ]:
# Verify accuracy
error_basic = np.sort(np.abs(np.sort(eig_basic) - np.sort(eig_numpy)))
error_shifted = np.sort(np.abs(np.sort(eig_shifted) - np.sort(eig_numpy)))

print("Absolute errors:")
print(f"Basic QR:   {error_basic}")
print(f"Shifted QR: {error_shifted}")
print(f"\nMax error (Basic):   {np.max(error_basic):.2e}")
print(f"Max error (Shifted): {np.max(error_shifted):.2e}")

## Convergence Visualization

We now visualize the convergence behavior of both algorithms, showing:
1. Evolution of diagonal elements (eigenvalue estimates)
2. Decay of off-diagonal elements

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Eigenvalue convergence (Basic QR)
ax1 = axes[0, 0]
hist_array_basic = np.array(hist_basic)
for i in range(n):
    ax1.plot(hist_array_basic[:, i], label=f'$\\lambda_{i+1}$', linewidth=1.5)
for eig in eig_numpy:
    ax1.axhline(y=eig, color='k', linestyle='--', alpha=0.3)
ax1.set_xlabel('Iteration', fontsize=12)
ax1.set_ylabel('Diagonal Elements', fontsize=12)
ax1.set_title('Basic QR: Eigenvalue Convergence', fontsize=14)
ax1.legend(loc='best', fontsize=9)
ax1.grid(True, alpha=0.3)

# Plot 2: Eigenvalue convergence (Shifted QR)
ax2 = axes[0, 1]
hist_array_shifted = np.array(hist_shifted)
for i in range(n):
    ax2.plot(hist_array_shifted[:, i], label=f'$\\lambda_{i+1}$', linewidth=1.5)
for eig in eig_numpy:
    ax2.axhline(y=eig, color='k', linestyle='--', alpha=0.3)
ax2.set_xlabel('Iteration', fontsize=12)
ax2.set_ylabel('Diagonal Elements', fontsize=12)
ax2.set_title('Shifted QR: Eigenvalue Convergence', fontsize=14)
ax2.legend(loc='best', fontsize=9)
ax2.grid(True, alpha=0.3)

# Plot 3: Off-diagonal decay comparison (log scale)
ax3 = axes[1, 0]
ax3.semilogy(off_basic, 'b-', label='Basic QR', linewidth=2)
ax3.semilogy(off_shifted, 'r-', label='Shifted QR', linewidth=2)
ax3.axhline(y=1e-10, color='k', linestyle=':', alpha=0.5, label='Tolerance')
ax3.set_xlabel('Iteration', fontsize=12)
ax3.set_ylabel('Off-diagonal Norm (log scale)', fontsize=12)
ax3.set_title('Convergence Rate Comparison', fontsize=14)
ax3.legend(loc='best', fontsize=10)
ax3.grid(True, alpha=0.3)

# Plot 4: Final matrix structure visualization
ax4 = axes[1, 1]

# Run a few iterations and show matrix structure
A_demo = A.copy()
for _ in range(10):
    Q, R = qr(A_demo)
    A_demo = R @ Q

im = ax4.imshow(np.abs(A_demo), cmap='viridis', aspect='equal')
ax4.set_title('Matrix Structure after 10 Iterations\n(absolute values)', fontsize=14)
ax4.set_xlabel('Column Index', fontsize=12)
ax4.set_ylabel('Row Index', fontsize=12)
cbar = plt.colorbar(im, ax=ax4)
cbar.set_label('|Value|', fontsize=10)

# Add text annotations for diagonal elements
for i in range(n):
    for j in range(n):
        if i == j:
            ax4.text(j, i, f'{A_demo[i,j]:.2f}', ha='center', va='center', 
                    color='white', fontsize=8, fontweight='bold')

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPlot saved to 'plot.png'")

## Performance Analysis

Let's analyze the computational efficiency of both methods.

In [ ]:
# Performance comparison across different matrix sizes
sizes = [4, 6, 8, 10]
iterations_basic = []
iterations_shifted = []

print("Performance Comparison:")
print("="*60)
print(f"{'Size':<10} {'Basic QR Iter':<20} {'Shifted QR Iter':<20}")
print("="*60)

for size in sizes:
    # Generate random symmetric matrix
    B_test = np.random.randn(size, size)
    A_test = (B_test + B_test.T) / 2
    
    # Basic QR
    _, hist_b, _ = qr_algorithm_basic(A_test, max_iterations=500, tol=1e-10)
    iter_basic = len(hist_b) - 1
    
    # Shifted QR
    _, hist_s, _ = qr_algorithm_shifted(A_test, max_iterations=500, tol=1e-10)
    iter_shifted = len(hist_s) - 1
    
    iterations_basic.append(iter_basic)
    iterations_shifted.append(iter_shifted)
    
    speedup = iter_basic / iter_shifted if iter_shifted > 0 else float('inf')
    print(f"{size:<10} {iter_basic:<20} {iter_shifted:<20} (speedup: {speedup:.1f}x)")

print("="*60)

## Conclusion

The QR algorithm is a powerful iterative method for computing matrix eigenvalues. Key takeaways:

1. **Basic QR Algorithm**: Converges linearly, with rate determined by the ratio of eigenvalue magnitudes

2. **Shifted QR Algorithm**: Dramatically accelerates convergence through the use of shifts, achieving cubic convergence for symmetric matrices with Rayleigh quotient shifts

3. **Practical Considerations**:
   - Modern implementations use Hessenberg reduction as preprocessing
   - Implicit QR methods avoid explicit matrix products
   - Deflation techniques handle converged eigenvalues efficiently

The QR algorithm, particularly with shifts and deflation, forms the backbone of eigenvalue routines in libraries like LAPACK, NumPy, and MATLAB.